Extracción

In [6]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread 
import pandas as pd
import numpy as np


In [7]:
#Documentación
ruta_permisos = "D:/SITEC/permisos/sitecsolucionesyservicios-67cee8fd8c65.json"          
alcances = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive'] 

#credenciales
cred = service_account.Credentials.from_service_account_file(ruta_permisos, scopes = alcances)

#Objeto de conexión
servicesheets = build('sheets','v4',credentials=cred)
servicedrive = build('drive','v3', credentials= cred)

gs = servicesheets.spreadsheets()

Tomaremos 3 cotizaciones y extraeremos su información para luego transformarla


In [ ]:
#Extracción de una cotización
#cotn[nombre,id]
cot1 = ["CTQ-1020 EATON BUSSMANN (SERVICIO DE RENTA DE PLATAFORMA TIJERA JLG MODELO 2030ES)","16h05uAcLjE7rqvFzD9Ij3PDMZcdg-bai5al5Ijz9r2o"]

nom_hoja = 'CARATULA'
ran = '!A1:H26'

extract1 = gs.values().get(
                spreadsheetId = cot1[1],
                range = nom_hoja+ran).execute()
        
       

#encabezados = extract1['values'][0]
data = extract1['values'][0:]
extraccion1 = pd.DataFrame(data)

print(extraccion1)

In [120]:
#extracción de 3 cotizaciones al mismo tiempo, con su respectivo nombre:
metadata = {'CTQ-1020 EATON BUSSMANN (SERVICIO DE RENTA DE PLATAFORMA TIJERA JLG MODELO 2030ES)' : '16h05uAcLjE7rqvFzD9Ij3PDMZcdg-bai5al5Ijz9r2o',
            'CTQ-1255 ZF (TRAMITOLOGIA Y GESTION DE INCREMENTOS DE CARGA Y DOCUMENTACION DE OBRAS ANTE CFE) (1)' : '15xpNv-lkcE5ufSMEtweBBAjUA0zv7Ghtv3f-xOlXLPE',
            'CTQ-1261 POLIGOF (DICTAMEN ELECTRICO)' : '1gY6FAEvBap8P7LqEPkemuFB2fBpXlpAhQEBtv_NOtvc'}

#definimos la estrctura que contendra los datos de la factura
#rango = 'A1:H26'

def extraccion(service, sp_id,range):
    res = gs.values().get(spreadsheetId = sp_id,range = nom_hoja + ran).execute()
    values = res.get('values', [])                  #convertimos los dataframes desde la función
    return pd.DataFrame(values) 
    

In [121]:
#Creamos un diccionario para guardar los dataframes.
cotizaciones = {}
# for ids,nom_sheet in metadata.items():
#     print(f"este es el nombre: {ids} y este es su id: {nom_sheet}")

#datos = extraccion(gs,metadata['CTQ-1020 EATON BUSSMANN (SERVICIO DE RENTA DE PLATAFORMA TIJERA JLG MODELO 2030ES)'],ran)
#print(datos)

for id,nombre in metadata.items():
    cotizaciones[nombre]= extraccion(gs,id,ran)
    




HttpError: <HttpError 404 when requesting https://sheets.googleapis.com/v4/spreadsheets/CTQ-1020%20EATON%20BUSSMANN%20%28SERVICIO%20DE%20RENTA%20DE%20PLATAFORMA%20TIJERA%20JLG%20MODELO%202030ES%29/values/CARATULA%21A1%3AH26?alt=json returned "Requested entity was not found.". Details: "Requested entity was not found.">